In [27]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re

In [11]:
# Testing 1
for page in range(1,2):
    link = 'https://www.apartments.com/houses/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='premiumPlus placard')
    pprint(results)

[<article class="premiumPlus placard" data-listingid="kcjywq8" data-url="https://www.apartments.com/20026-kittridge-st-los-angeles-ca/kcjywq8/">
<header class="placardHeader">
<a class="placardTitle js-placardTitle " href="https://www.apartments.com/20026-kittridge-st-los-angeles-ca/kcjywq8/" title="20026 Kittridge St, Los Angeles, CA">
20026 Kittridge St</a>
<div class="location" title="Los Angeles, CA 91306">Los Angeles, CA 91306</div>
</header>
<section class="placardContent">
<div class="media">
<div class="imageContainer carousel slide" data-interval="false">
<div class="carouselInner">
<div class="item active " style='background-image: url("https://images1.apartments.com/i2/c7x7JVYaz1bi0wABfhQDPoFI5oEvb8l1aviSAgN5ZzM/117/20026-kittridge-st-los-angeles-ca-building-photo.jpg");' title="Building Photo - 20026 Kittridge St">
</div>
</div>
</div>
</div>
<div class="propertyInfo">
<div class="infoPadding">
<div class="freshnessAndFavoritesWrapper">
<span class="listingFreshness">
<span

In [28]:
rent_obj = []
all_property = []

for page in range(24,29):
    link = 'https://www.apartments.com/houses/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='prosumer placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
        find_tag = result.find('a')['href']
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "location").text
        city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        lease_length = house_details.find('td', class_="leaseLength").text
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Houses",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
print(all_property)

[{'City_Rent': 'Fairfield', 'Type': 'Houses', 'State_Rent': 'CA', 'Zip_code': '94533', 'Bedroom_Info': [{'bedroom_type': '3 Beds', 'price': '$2,200'}], 'Full_Address': 'Fairfield, CA 94533', 'Rent_link': 'https://www.apartments.com/237-larkspur-ln-fairfield-ca/mspqznj/', 'Bath_Info': '2 BAs', 'Square_Feet_Data': '1,405 Sq Ft', 'Square_Feet': '1,405', 'Lease_length': '12 Month Lease', 'Availability_check': 'May 1', '3 Beds': '$2,200'}, {'City_Rent': '2432 Middlebury Dr', 'Type': 'Houses', 'State_Rent': 'Lathrop', 'Zip_code': 'CA', 'Bedroom_Info': [{'bedroom_type': '4+ Beds', 'price': '$2,795'}], 'Full_Address': 'Lathrop, CA 95330', 'Rent_link': 'https://www.apartments.com/2432-middlebury-dr-lathrop-ca/vx9s92d/', 'Bath_Info': '4½ BAs', 'Square_Feet_Data': '2,317 Sq Ft', 'Square_Feet': '2,317', 'Lease_length': '', 'Availability_check': 'Available Now', '4+ Beds': '$2,795'}, {'City_Rent': "7261 O'Donovan Rd", 'Type': 'Houses', 'State_Rent': 'Creston', 'Zip_code': 'CA', 'Bedroom_Info': [{'b

In [25]:
rent_data = pd.DataFrame(all_property)
rent_data

,1 Bed,2 Beds,3 Beds,4 Beds,Availability_check,Bath_Info,Bedroom_Info,City_Rent,Full_Address,Lease_length,Rent_link,Square_Feet,Square_Feet_Data,State_Rent,Type,Zip_code
0,NaN,NaN,NaN,"$4,700",Available Now,3½ BAs,"[{'bedroom_type': '4 Beds', 'price': '$4,700'}]",Monterey,"Monterey, CA 93940",12 Month Lease,https://www.apartments.com/23-cramden-dr-monte...,"3,300","3,300 Sq Ft",CA,Houses,93940
1,NaN,NaN,"$2,650",NaN,Available Now,2 BAs,"[{'bedroom_type': '3 Beds', 'price': '$2,650'}]",San Gabriel,"330 W Marshall St, San Gabriel, CA 91776",,https://www.apartments.com/330-w-marshall-st-s...,"1,291","1,291 Sq Ft",CA,Houses,91776
2,NaN,NaN,NaN,"$2,250",Available Now,2 BAs,"[{'bedroom_type': '4 Beds', 'price': '$2,250'}]",Palmdale,"Palmdale, CA 93550",12 Month Lease,https://www.apartments.com/2019-sweetbrier-st-...,"1,400","1,400 Sq Ft",CA,Houses,93550
3,NaN,NaN,"$8,900",NaN,Available Now,3½ BAs,"[{'bedroom_type': '3 Beds', 'price': '$8,900'}]",Manhattan Beach,"216 29th Pl, Manhattan Beach, CA 90266",12 Month Lease,https://www.apartments.com/216-29th-pl-manhatt...,"2,300","2,300 Sq Ft",CA,Houses,90266
4,NaN,NaN,"$2,750",NaN,Available Now,2 BAs,"[{'bedroom_type': '3 Beds', 'price': '$2,750'}]",Rohnert Park,"Rohnert Park, CA 94928",12 Month Lease,https://www.apartments.com/974-hawthorne-cir-r...,"1,800","1,800 Sq Ft",CA,Houses,94928
5,"$1,250",NaN,NaN,NaN,Available Now,1 BA,"[{'bedroom_type': '1 Bed', 'price': '$1,250'}]",Hawthorne,"4249 W 136th St, Hawthorne, CA 90250",12 Month Lease,https://www.apartments.com/4249-w-136th-st-haw...,230,230 Sq Ft,CA,Houses,90250
6,NaN,NaN,"$6,000",NaN,Available Now,2 BAs,"[{'bedroom_type': '3 Beds', 'price': '$6,000'}]",Saratoga,"Saratoga, CA 95070",12 Month Lease,https://www.apartments.com/18394-montpere-way-...,"1,700","1,700 Sq Ft",CA,Houses,95070
7,NaN,"$2,250",NaN,NaN,Available Now,1 BA,"[{'bedroom_type': '2 Beds', 'price': '$2,250'}]",Long Beach,"Long Beach, CA 90805",,https://www.apartments.com/6240-myrtle-ave-lon...,861,861 Sq Ft,CA,Houses,90805
8,NaN,NaN,"$3,695",NaN,Available Now,3 BAs,"[{'bedroom_type': '3 Beds', 'price': '$3,695'}]",Glendale,"Glendale, CA 91203",12 Month Lease,https://www.apartments.com/623-salem-st-glenda...,"1,600","1,600 Sq Ft",CA,Houses,91203
9,NaN,"$6,200",NaN,NaN,Available Now,1 BA,"[{'bedroom_type': '2 Beds', 'price': '$6,200'}]",Los Angeles,"17065 Bollinger Dr, Los Angeles, CA 90272",12 Month Lease,https://www.apartments.com/17065-bollinger-dr-...,"1,286","1,286 Sq Ft",CA,Houses,90272


In [26]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q1-2020_Apartment.com_Houses_Data_page24.csv", index=False, header=True)